## Superstore Challenge

The superstore dataset is a collection of ~10,000 anonymized transactions from an e-commerce store that occurred between the years 2014 and 2017.  

Your task here is to make a 'meta' dataset that aggregates information about what individual customers have done over their lifetime of interactions with the store.  

When you're done you'll have a list of each unique customer in the store (going by their name), with information like their lifetime customer value, number of orders, and their ordering behavior measured over different lags of time.  

This will be a helpful exercise to re-inforce some of the concepts discussed from class 6, like grouping and date offsets.

In [547]:
import pandas as pd
import numpy as np

ss = pd.read_excel('/Users/swllms/DAT-10-14-SW/class material/Unit2/data/superstore.xls')

**Column 1:** Create a column that lists every customers lifetime customer value.

In [548]:
cust_life = 1/cr ###Customer Lifetime = 1/Churn Rate

In [505]:
total_cust = ss['Customer ID'].nunique()
total_orders = ss.groupby('Customer Name')['Order ID'].nunique()
pur_freq = total_orders / total_cust ###Purchase Frequency =  Total Number of Orders / Total Number of Customers
total_rev = ss['Sales'].sum()
avo = total_rev/total_orders ###Average Order Value = Total Revenue / Total Number of Orders
prof_mar = ss['Profit'].sum()
cust_val = avo/pur_freq ###Customer Value = Average Order Value * Purchase Frequency
rr = (total_orders > 1) / total_orders ### number of orders > 1/ number of orders = repeat rate
cr = 1-rr ###Churn Rate = 1-Repeat Rate
###CLTV = ((Average Order Value x Purchase Frequency)/Churn Rate) x Profit margin
cltv = ((avo * pur_freq)/cr) * prof_mar
cltv.head()

Customer Name
Aaron Bergman      1.244473e+09
Aaron Hawkins      9.679236e+08
Aaron Smayling     9.679236e+08
Adam Bellavance    9.481700e+08
Adam Hart          9.218320e+08
Name: Order ID, dtype: float64

In [549]:
clv = ss.groupby('Customer Name')['Sales'].sum().to_frame()
##cust_life_val = cust_life_val.to_frame().rename({'Sales': 'CLV'}, axis=1)
###sales = ss.merge(cust_life_val, on='Customer Name', how='left')
clv.head()

,Sales
Customer Name,
Aaron Bergman,886.156
Aaron Hawkins,1744.700
Aaron Smayling,3050.692
Adam Bellavance,7755.620
Adam Hart,3250.337


**Column 2:** Create a column that lists the length of time each customer has been with the store.  This is defined as the number of days between when they made their first purchase and today.

In [455]:
### Merged it to the main colm
#first_ord = ss.groupby('Customer Name')['Order Date'].min()
#cust_duration = np.datetime64('now') - first_ord
#cust_duration = cust_duration.to_frame().rename({'Order Date': 'Customer Duration'}, axis=1)
#ss = ss.merge(cust_duration, on='Customer Name', how='left')
#ss.head()

In [550]:
# length of time customer has been with us
first_ord = ss.groupby(['Customer Name'])['Order Date'].min()
lot = np.datetime64('now') - first_ord
clv = clv.merge(lot.to_frame(), on='Customer Name')
clv.rename({'Order Date': 'Customer Duration'}, axis=1, inplace=True)
clv.head()

,Sales,Customer Duration
Customer Name,,
Aaron Bergman,886.156,2085 days 21:15:33
Aaron Hawkins,1744.700,2022 days 21:15:33
Aaron Smayling,3050.692,1926 days 21:15:33
Adam Bellavance,7755.620,1508 days 21:15:33
Adam Hart,3250.337,1814 days 21:15:33


**Column 3:** Create a column that lists the total number of purchases for each customer.

In [457]:
###Merged to main dataframe
#cust_tot_pur = ss.groupby('Customer ID')['Order ID'].nunique(sum)
#cust_tot_pur = cust_tot_pur.to_frame().rename({'Order ID': 'All Orders'}, axis=1)
#ss = ss.merge(cust_tot_pur, on='Customer ID', how='left')

In [551]:
cust_tot_pur = ss.groupby('Customer Name')['Order ID'].nunique(sum)
clv = clv.merge(cust_tot_pur.to_frame(), on='Customer Name')
clv.rename({'Order ID': 'Total Purchases'}, axis=1, inplace=True)
clv.head()

,Sales,Customer Duration,Total Purchases
Customer Name,,,
Aaron Bergman,886.156,2085 days 21:15:33,3
Aaron Hawkins,1744.700,2022 days 21:15:33,7
Aaron Smayling,3050.692,1926 days 21:15:33,7
Adam Bellavance,7755.620,1508 days 21:15:33,8
Adam Hart,3250.337,1814 days 21:15:33,10


**Column 4:** Create a column that assigns customers to a cohort.  

A customers cohort is determined by when they made their first purchase, and every year in your dataset has two cohorts:  the first half of the year and the second half.  

For example, if someone made their first purchase in March of 2017, their cohort would be 2017-1, or something similar.  Someone who purchased in September of 2017 would be 2017-2, and so on.

In [552]:
#first_ord = ss.groupby('Customer Name')['Order Date'].min()

date_range =[
    first_ord.between('01/01/2014', '06/30/2014'),
    first_ord.between('07/01/2014', '12/31/2014'),
    first_ord.between('01/01/2015', '06/30/2015'),
    first_ord.between('07/01/2015', '12/31/2015'),    
    first_ord.between('01/01/2016', '06/30/2016'),
    first_ord.between('07/01/2016', '12/31/2016'),
    first_ord.between('01/01/2017', '06/30/2017'),
    first_ord.between('07/01/2017', '12/31/2017')
]

group = [
    '1H-14', 
    '2H-14', 
    '1H-15', 
    '2H-15', 
    '1H-16', 
    '2H-16', 
    '1H-17', 
    '2H-17'
]
clv['Cohort'] = np.select(date_range, group, 'Other')
clv.head()

,Sales,Customer Duration,Total Purchases,Cohort
Customer Name,,,,
Aaron Bergman,886.156,2085 days 21:15:33,3,1H-14
Aaron Hawkins,1744.700,2022 days 21:15:33,7,1H-14
Aaron Smayling,3050.692,1926 days 21:15:33,7,2H-14
Adam Bellavance,7755.620,1508 days 21:15:33,8,2H-15
Adam Hart,3250.337,1814 days 21:15:33,10,2H-14


In [510]:
### Could not merge this to main df because of 'numpy.ndarray' object has no attribute 'to frame'
#cohort = cohort.to_frame().rename({'Order Date': 'Cohort'}, axis=1)
#ss = ss.merge(cohort, on='Customer Name', how='left')

**Column 5:** Create a column that lists whether or not they're a repeat customer.  This means they've made more than one order.

In [495]:
### My answer resulted in a new colm titled order id instead of repeat customer. 
#total_orders = clv.groupby('Customer Name')['Total Purchases'].nunique(sum)
#repeat_cust = total_orders > 1
#clv = clv.merge(cust_tot_pur.to_frame(), on='Customer Name')
#clv.rename({'Total Purchases': 'Repeat Customer'}, axis=1)
#clv.head()

In [553]:
clv['Repeat Customer'] = np.where(clv['Total Purchases'] > 1, True, False)
clv.head()

,Sales,Customer Duration,Total Purchases,Cohort,Repeat Customer
Customer Name,,,,,
Aaron Bergman,886.156,2085 days 21:15:33,3,1H-14,True
Aaron Hawkins,1744.700,2022 days 21:15:33,7,1H-14,True
Aaron Smayling,3050.692,1926 days 21:15:33,7,2H-14,True
Adam Bellavance,7755.620,1508 days 21:15:33,8,2H-15,True
Adam Hart,3250.337,1814 days 21:15:33,10,2H-14,True


**Column 6:** We want to find out what type of customer each person is.  To do this, we want to find which value of 'Segment' occurred most frequently for every single customer.  Ie, the modal value of the 'Segment' column for each customer.

In [431]:
###merged to maine df
#seg_freq = ss.groupby('Customer Name')['Segment'].agg(pd.Series.mode)
#seg_freq = seg_freq.to_frame().rename({'Segment': 'Segment Mode'}, axis=1)
#ss = ss.merge(seg_freq, on='Customer Name', how='left')

In [554]:
seg_freq = ss.groupby('Customer Name')['Segment'].agg(pd.Series.mode)
clv = clv.merge(seg_freq.to_frame(), on='Customer Name')
clv.head()

,Sales,Customer Duration,Total Purchases,Cohort,Repeat Customer,Segment
Customer Name,,,,,,
Aaron Bergman,886.156,2085 days 21:15:33,3,1H-14,True,Consumer
Aaron Hawkins,1744.700,2022 days 21:15:33,7,1H-14,True,Corporate
Aaron Smayling,3050.692,1926 days 21:15:33,7,2H-14,True,Corporate
Adam Bellavance,7755.620,1508 days 21:15:33,8,2H-15,True,Home Office
Adam Hart,3250.337,1814 days 21:15:33,10,2H-14,True,Corporate


**Column 7:** Create a column that is the date *3 months after they made their first purchase*.

In [555]:
three_mo_after = pd.DateOffset(months=3)

In [556]:
three_mon_after_1_pur = ss.groupby('Customer Name')['Order Date'].min() + three_mo_after
three_mon_after_1_pur = three_mon_after_1_pur.to_frame().rename({'Order Date': '3M After Purchase 1'}, axis=1)
clv = clv.merge(three_mon_after_1_pur, on='Customer Name')
ss = ss.merge(three_mon_after_1_pur, on='Customer Name')
clv.head(2)

,Sales,Customer Duration,Total Purchases,Cohort,Repeat Customer,Segment,3M After Purchase 1
Customer Name,,,,,,,
Aaron Bergman,886.156,2085 days 21:15:33,3,1H-14,True,Consumer,2014-05-18
Aaron Hawkins,1744.700,2022 days 21:15:33,7,1H-14,True,Corporate,2014-07-22


**Column 8:** Make a column that represents one year after they made their first purchase.  Add this to your initial dataframe as well.

In [559]:
one_year = pd.DateOffset(years=1)
clv['1YR After Purchase 1'] = first_ord + one_year
clv.head()

,Sales,Customer Duration,Total Purchases,Cohort,Repeat Customer,Segment,3M After Purchase 1,1YR After Purchase 1
Customer Name,,,,,,,,
Aaron Bergman,886.156,2085 days 21:15:33,3,1H-14,True,Consumer,2014-05-18,2015-02-18
Aaron Hawkins,1744.700,2022 days 21:15:33,7,1H-14,True,Corporate,2014-07-22,2015-04-22
Aaron Smayling,3050.692,1926 days 21:15:33,7,2H-14,True,Corporate,2014-10-27,2015-07-27
Adam Bellavance,7755.620,1508 days 21:15:33,8,2H-15,True,Home Office,2015-12-18,2016-09-18
Adam Hart,3250.337,1814 days 21:15:33,10,2H-14,True,Corporate,2015-02-16,2015-11-16


In [561]:
ss = ss.merge(clv['1YR After Purchase 1'].to_frame(), on='Customer Name', how='left')
ss.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,3M After Purchase 1,1YR After Purchase 1
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960,2,0.0,41.9136,2016-01-15,2016-10-15
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.940,3,0.0,219.5820,2016-01-15,2016-10-15
2,5492,CA-2017-164098,2017-01-26,2017-01-27,First Class,CG-12520,Claire Gute,Consumer,United States,Houston,...,OFF-ST-10000615,Office Supplies,Storage,"SimpliFile Personal File, Black Granite, 15w x...",18.160,2,0.2,1.8160,2016-01-15,2016-10-15
3,6878,US-2015-123918,2015-10-15,2015-10-15,Same Day,CG-12520,Claire Gute,Consumer,United States,Dallas,...,FUR-FU-10004952,Furniture,Furnishings,C-Line Cubicle Keepers Polyproplyene Holder w/...,131.376,6,0.6,-95.2476,2016-01-15,2016-10-15
4,6879,US-2015-123918,2015-10-15,2015-10-15,Same Day,CG-12520,Claire Gute,Consumer,United States,Dallas,...,OFF-PA-10003001,Office Supplies,Paper,Xerox 1986,5.344,1,0.2,1.8704,2016-01-15,2016-10-15


**Column 9:** Make a column that determines whether or not a customer made a second purchase within 90 days of their first purchase.

In [562]:
first_ord = ss.groupby('Customer Name')['Order Date'].min()
first_ord = first_ord.to_frame().rename({'Order Date': 'First Order'}, axis=1)
ss = ss.merge(first_ord, on='Customer Name', how='left')

In [563]:
ss['3M Repeat'] = ss['Order Date'].between(ss['First Order'], ss['3M After Purchase 1'], inclusive=False)
tm_repeats = ss.groupby('Customer Name')['3M Repeat'].any()
clv['3M RepPur'] = tm_repeats
clv.head()

,Sales,Customer Duration,Total Purchases,Cohort,Repeat Customer,Segment,3M After Purchase 1,1YR After Purchase 1,3M RepPur
Customer Name,,,,,,,,,
Aaron Bergman,886.156,2085 days 21:15:33,3,1H-14,True,Consumer,2014-05-18,2015-02-18,True
Aaron Hawkins,1744.700,2022 days 21:15:33,7,1H-14,True,Corporate,2014-07-22,2015-04-22,True
Aaron Smayling,3050.692,1926 days 21:15:33,7,2H-14,True,Corporate,2014-10-27,2015-07-27,False
Adam Bellavance,7755.620,1508 days 21:15:33,8,2H-15,True,Home Office,2015-12-18,2016-09-18,False
Adam Hart,3250.337,1814 days 21:15:33,10,2H-14,True,Corporate,2015-02-16,2015-11-16,False


**Column 10:** Make a column that counts how many items each customer made with in their first year of purchase.

In [564]:
ss['1YR Purchases'] = ss['Order Date'].between(ss['First Order'], ss['1YR After Purchase 1'])
y_1_purchases = (ss.groupby(['Customer Name', 'Order ID'])['1YR Purchases'].sum() > 0).sum(level='Customer Name').to_frame()

In [566]:
clv = clv.merge(y_1_purchases, on='Customer Name')
clv.head()

,Sales,Customer Duration,Total Purchases,Cohort,Repeat Customer,Segment,3M After Purchase 1,1YR After Purchase 1,3M RepPur,1YR Purchases_x,1YR Purchases_y
Customer Name,,,,,,,,,,,
Aaron Bergman,886.156,2085 days 21:15:33,3,1H-14,True,Consumer,2014-05-18,2015-02-18,True,2.0,2.0
Aaron Hawkins,1744.700,2022 days 21:15:33,7,1H-14,True,Corporate,2014-07-22,2015-04-22,True,4.0,4.0
Aaron Smayling,3050.692,1926 days 21:15:33,7,2H-14,True,Corporate,2014-10-27,2015-07-27,False,1.0,1.0
Adam Bellavance,7755.620,1508 days 21:15:33,8,2H-15,True,Home Office,2015-12-18,2016-09-18,False,4.0,4.0
Adam Hart,3250.337,1814 days 21:15:33,10,2H-14,True,Corporate,2015-02-16,2015-11-16,False,1.0,1.0


**Column 11:** Make a column that sums up the total value of sales for each customer one year after they made their purchase.

SyntaxError: invalid syntax (<ipython-input-568-f29ae3b3a43f>, line 1)

**Hint:** For some of these columns, you'll need to break them down into a few steps.  It's okay to make  helper columns on your original dataset that make it easier for you to calculate the final result.

### Questions:

Now that you've made these columns, try and answer the following questions.

**What percentage of customers make a second purchase within 3 months after their first one?  How does this differ by customer segment?**

**How Has the 3-Month Repurchasing Rate Been Changing Across Cohorts?**

**What impact does a second order within 3 months of 1st purchase have on lifetime customer value?  Does this effect hold true for each customer segment?**

**What's the average expected sales value for a customer one year after their first purchase? How has this changed across cohorts?**

**How much does lifetime customer value differ across the different customer segments?**